## Configurações Pyspark

In [20]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import Window

import pandas as pd

def write_to_postgresl(df, tb_name=None, write_mode='None'):
    df.count()
    if tb_name is None:
        raise Exception('Informe o nome da tabela')
    if write_mode is None:
        raise Exception('Informe o mode de escrita: append ou overwrite')
        
    from datetime import datetime
    start_time = datetime.now()
    
    try:
        df.write.jdbc(url, table=tb_name, mode=write_mode, properties=properties)
    except Exception as e:
        print(f'Erro: {e}')
    end_time = datetime.now()
    
    total_time = str(end_time - start_time)
    
    return f'Total time: {total_time} and - Total rows: {df.count()} - Total columns: {len(df.columns)}'



    # Criando a sessão do Spark
spark  = SparkSession.builder \
    .appName("Data Analysis") \
    .config('spark.jars', '/data/IDAF/DATABASECONNECTOR_JAR_FOLDER/postgresql-42.2.18.jar')\
    .config("spark.executor.memory", "8g")\
    .config("spark.driver.memory", "8g")\
    .config("spark.executor.cores", "4")\
    .config("spark.executor.instances","8")\
    .config("spark.sql.shuffle.partitions","96")\
    .config("spark.default.parallelism","96")\
    .getOrCreate()


url = "jdbc:postgresql://localhost:5432/postgres"

properties = {
    "user" : "postgres",
    "password" : "cidacs",
    "driver" : "org.postgresql.Driver"
}



In [21]:
import re
from pyspark.sql.types import ArrayType, StringType

def dividir_em_blocos(s):
    return re.findall('.{1,4}', s) if s else None

dividir_udf = F.udf(dividir_em_blocos, ArrayType(StringType()))

## Configurações Pandas

In [22]:
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100)
spark.conf.set("spark.sql.repl.eagerEval.maxNumRows", 300)

spark.conf.set("spark.sql.debug.maxToStringFields", 100)

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_rows", None)

### Lendo dados enriquecidos

In [23]:
df_input = (spark
            .read
            .parquet('/data/IDAF/PROJETOS/PARCERIA_CIDACS_PHDC/scripts_omop_tb/basefinal_tb_limp_enriched')
           )

In [24]:
df_vocab_map = spark.read.csv('/data/IDAF/PROJETOS/PARCERIA_CIDACS_PHDC/scripts_omop_tb/cte_vocab_map_standard.csv', header=True, sep='\t')

df_vocab_map.createOrReplaceTempView('cte_vocab_map_standard')
df_vocab_map.count()

5284745

In [25]:
df_vocab_map_standard = spark.sql("""
        SELECT 
            REPLACE(source_code, '.', '') as source_code,
            target_concept_id,
            source_vocabulary_id,
            source_concept_id
        FROM cte_vocab_map_standard csv
        WHERE true 
        and target_domain_id = 'Condition'
        and target_standard_concept = 'S'
        and source_vocabulary_id = 'ICD10'
""")
df_vocab_map_standard.count()

15762

In [26]:
df_vocab_map_source = spark.sql("""
        SELECT 
            REPLACE(source_code, '.', '') as source_code,
            source_concept_id            
        FROM cte_vocab_map_standard csv2
        WHERE source_vocabulary_id = 'ICD10'
""")
df_vocab_map_source.count()

19103

### Condition Occurrence

In [27]:
window_condition_occurrence = Window.partitionBy().orderBy('person_id_infant', 'dtnasc_sinasc')

In [28]:
df_condition_occurrence = (
df_input
    .withColumn('person_id', F.col('person_id_infant'))
    .withColumn('idanomal_sinasc_code', F.when(F.col('idanomal_sinasc').isin(0,2,88), 0)
                                         .when(F.col('idanomal_sinasc')==1, F.col('codanomal_sinasc'))
                                         .otherwise(0))
    .withColumn('tpapresent_sinasc_code', F.when(F.col('tpapresent_sinasc')==1, 4312344)
                                           .when(F.col('tpapresent_sinasc')==2, 4195566)
                                           .when(F.col('tpapresent_sinasc')==3, 81358)
                                           .when(F.col('tpapresent_sinasc').isin(9,88), 0)
                                           .otherwise(0))
    .withColumn('concept', F.split(F.concat_ws(',', 'idanomal_sinasc_code', 'tpapresent_sinasc_code', 'codanomal_sinasc'), ','))
    .withColumn('concept_id_exploded', F.explode(F.col('concept')))
    .join(df_vocab_map_standard, F.col('concept_id_exploded')==df_vocab_map_standard.source_code, how='left')
    .join(df_vocab_map_source, F.col('concept_id_exploded')==df_vocab_map_source.source_code, how='left')
    .withColumn('condition_concept_id', F.col('target_concept_id'))
    .withColumn('condition_source_concept_id', F.col('csv.source_concept_id'))
    .withColumn('condition_start_date', F.col('dtnasc_sinasc'))
    .withColumn('condition_type_concept_id', F.lit(32879))
    .withColumn('condition_start_datetime', F.lit(None))
    .withColumn('condition_end_date', F.lit(None))
    .withColumn('condition_end_datetime', F.lit(None))
    .withColumn('stop_reason', F.lit(None))
    .withColumn('condition_status_concept_id', F.lit(None))
    .withColumn('provider_id', F.lit(None))
    .withColumn('visit_occurrence_id', F.col('visit_occurrence_id_infant'))
    .withColumn('visit_detail_id', F.lit(None))
    .withColumn('condition_source_value', F.col('concept_id_exploded'))
    .withColumn('condition_status_source_value', F.lit(None))
    .withColumn('condition_occurrence_id', F.row_number().over(window_condition_occurrence)+F.lit('1000000000'))
    .withColumn('condition_concept_id', F.when((F.col('condition_source_concept_id').isNull())
                                                     &(F.col('condition_source_value').isin('0', '4312344', '4195566', '81358'))
                                                     ,F.col('condition_source_value')).otherwise(F.col('condition_concept_id')))        
    .select(
                F.col('condition_occurrence_id').cast('integer'),
                F.col('person_id').cast('integer'),
                F.col('condition_concept_id').cast('integer'), # Snomed
                F.col('condition_start_date').cast('date'),
                F.col('condition_start_datetime').cast('timestamp'),
                F.col('condition_end_date').cast('date'),
                F.col('condition_end_datetime').cast('timestamp'),
                F.col('condition_type_concept_id').cast('integer'),
                F.col('condition_status_concept_id').cast('integer'),
                F.col('stop_reason').cast('string'),
                F.col('provider_id').cast('integer'),
                F.col('visit_occurrence_id').cast('integer'),
                F.col('visit_detail_id').cast('integer'),
                F.col('condition_source_value').cast('string'),
                F.col('condition_source_concept_id').cast('integer'), # CID10
                F.col('condition_status_source_value').cast('string'))
    
    .filter(F.col('condition_concept_id').isNotNull())
    .filter(F.col('condition_concept_id')!=0)
)


df_condition_occurrence.count()

15797952

In [30]:
write_to_postgresl(df_condition_occurrence, tb_name='tuberculose.condition_occurrence', write_mode='append')

25/07/16 23:10:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/07/16 23:10:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/07/16 23:10:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/07/16 23:11:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/07/16 23:11:00 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/07/16 23:11:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/07/16 2

'Total time: 6:27:00.741697 and - Total rows: 15797952 - Total columns: 16'